In [1]:
import pandas as pd
import numpy as np
import jieba
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def get_deafult():
    return defaultdict(lambda: 3)
table = defaultdict(get_deafult)

In [4]:
with open('../data/train.csv', encoding='utf-8') as f:
    f.readline()
    for line in f.readlines():
        content_id, content, subject, sentiment_value, sentiment_word = line.split(',')
        table['content'][content_id] = content
        table[subject][content_id] = int(sentiment_value)+1
        word = 'word_{}'.format(subject)
        # table[word][content_id] = sentiment_word

In [5]:
df = pd.DataFrame.from_dict(table)

In [6]:
df.head(2)

,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.fillna(3, inplace=True)
df.rename_axis('content_id', inplace=True)

In [8]:
df.head(2)

,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
content_id,,,,,,,,,,,
01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [9]:
df.to_csv('../data/input.csv')

In [4]:

df = pd.read_csv('../data/train.csv')

df.groupby('subject').describe()

sentiment_value                                             
                  count      mean       std  min  25%  50%  75%  max
subject                                                             
价格               1273.0 -0.024352  0.450581 -1.0  0.0  0.0  0.0  1.0
内饰                536.0 -0.065299  0.700753 -1.0 -1.0  0.0  0.0  1.0
动力               2732.0  0.002196  0.528218 -1.0  0.0  0.0  0.0  1.0
外观                489.0  0.008180  0.680476 -1.0  0.0  0.0  0.0  1.0
安全性               573.0  0.012216  0.580744 -1.0  0.0  0.0  0.0  1.0
操控               1036.0  0.175676  0.620135 -1.0  0.0  0.0  1.0  1.0
油耗               1082.0  0.012015  0.516914 -1.0  0.0  0.0  0.0  1.0
空间                442.0  0.196833  0.679929 -1.0  0.0  0.0  1.0  1.0
舒适性               931.0 -0.155747  0.608556 -1.0 -1.0  0.0  0.0  1.0
配置                853.0 -0.039859  0.565690 -1.0  0.0  0.0  0.0  1.0

In [ ]:

word_set = set()


# In[7]:


for line in df['content']:
    words = jieba.cut(line)
    word_set.update(words)


# In[10]:


in_count = 0
with open('../data/merge_sgns_bigram_char300.txt', encoding='utf-8') as f, open('../data/cliped_vec.vec', 'w', encoding='utf-8') as outf:
    f.readline()
    while True:
        line = f.readline()
        if not line:
            break
        try:
            word, vec = line.split(' ', 1)
        except:
            print(line)
            continue
        if word in word_set:
            in_count += 1
            outf.write(line)
        else:
            print(word)
print('total have ', in_count, 'words in word embedding')


In [10]:
df = pd.read_csv('../data/input.csv')
df.head(5)

,content_id,content,价格,内饰,动力,外观,安全性,操控,油耗,空间,舒适性,配置
0,01DqbWMwRjfBxLYz,自动启停对车没好处，等红灯挂N挡，轻刹车。,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0
1,01X4vSqHci6NPYBy,开的2.0？在西藏动力如何？,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,02VHu4amZkpAGTb7,常用配件价格不贵，跟大众差不多。而且常用配件都有备货，如果不太常用的可能需要等，但也绝不是从...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,038JPMTIu1tRcZ2p,4万公里，悲催的11。。。市区每天办事，堵成汪。 哈哈哈。如果以前的锐志，起码13~14,3.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0
4,03AUEJcWODxb9ZHa,建议最好使用原厂，subaru的发动机的冷却方式更别的车不一样，所以原厂更适合。,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [11]:
def load_stopwords(filepath):
    with open(filepath, encoding='utf-8') as f:
        stops = set([v.strip() for v in f.readlines()])
    return stops
stops = load_stopwords('../data/stopwords.txt')
def seg_stop_merge(text):
    words = set(jieba.cut(text))
    words -= stops
    text = ' '.join(list(words))
    return text

In [12]:
corpus = df['content'].apply(lambda text: seg_stop_merge(text))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.960 seconds.
Prefix dict has been built succesfully.


In [13]:
vectorizer = CountVectorizer()

In [14]:
X = vectorizer.fit_transform(corpus)

In [15]:
word = vectorizer.get_feature_names()

In [16]:
word

['000',
 '01',
 '020',
 '03',
 '030',
 '04',
 '040',
 '05',
 '06',
 '07',
 '08',
 '09',
 '0a78d51926ec16809infoc',
 '0l',
 '0t',
 '0w',
 '0w20',
 '0w21',
 '0w2o',
 '0w30',
 '0w40',
 '10',
 '100',
 '1000',
 '10000',
 '100km',
 '1010',
 '10100',
 '102140',
 '1024x600',
 '103',
 '104',
 '1040',
 '105',
 '105000',
 '107',
 '108',
 '1080',
 '109',
 '10cm',
 '10km20',
 '10w',
 '10wkm',
 '10x',
 '11',
 '110',
 '1100',
 '110km',
 '1111',
 '115',
 '1150',
 '117',
 '118',
 '11ej25',
 '11l',
 '11x',
 '12',
 '120',
 '1200',
 '12000',
 '120km',
 '121',
 '1213',
 '125',
 '126',
 '128',
 '129',
 '12l',
 '12s',
 '12v',
 '13',
 '130',
 '1300',
 '13000',
 '1300km',
 '130km',
 '135',
 '135000',
 '136',
 '137mm',
 '138',
 '13800',
 '13l',
 '13xt',
 '14',
 '140',
 '1400',
 '14000',
 '145',
 '145000',
 '148',
 '14l',
 '14v',
 '15',
 '150',
 '1500',
 '15000',
 '150000',
 '15000km',
 '150p',
 '152',
 '15208aa160',
 '1528785224421',
 '153',
 '156as',
 '1580',
 '1582866622645548980',
 '15cm',
 '15kg',
 '15w',
 

In [17]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer

In [19]:
transformer = TfidfTransformer()
transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [20]:
tfidf = transformer.fit_transform(X)

In [21]:
tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
term_weights = []
word = vectorizer.get_feature_names()#获取词袋模型中的所有词语
weight = tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重
    print("-------这里输出第",i,u"类文本的词语tf-idf权重------")
    for j in range(len(word)):
        # print(word[j],weight[i][j])
        if weight[i][j] > 0:
            term_weights.append((word[j], weight[i][j]))


-------这里输出第 0 类文本的词语tf-idf权重------
-------这里输出第 1 类文本的词语tf-idf权重------
-------这里输出第 2 类文本的词语tf-idf权重------
-------这里输出第 3 类文本的词语tf-idf权重------
-------这里输出第 4 类文本的词语tf-idf权重------
-------这里输出第 5 类文本的词语tf-idf权重------
-------这里输出第 6 类文本的词语tf-idf权重------
-------这里输出第 7 类文本的词语tf-idf权重------
-------这里输出第 8 类文本的词语tf-idf权重------
-------这里输出第 9 类文本的词语tf-idf权重------
-------这里输出第 10 类文本的词语tf-idf权重------
-------这里输出第 11 类文本的词语tf-idf权重------
-------这里输出第 12 类文本的词语tf-idf权重------
-------这里输出第 13 类文本的词语tf-idf权重------
-------这里输出第 14 类文本的词语tf-idf权重------
-------这里输出第 15 类文本的词语tf-idf权重------
-------这里输出第 16 类文本的词语tf-idf权重------
-------这里输出第 17 类文本的词语tf-idf权重------
-------这里输出第 18 类文本的词语tf-idf权重------
-------这里输出第 19 类文本的词语tf-idf权重------
-------这里输出第 20 类文本的词语tf-idf权重------
-------这里输出第 21 类文本的词语tf-idf权重------
-------这里输出第 22 类文本的词语tf-idf权重------
-------这里输出第 23 类文本的词语tf-idf权重------
-------这里输出第 24 类文本的词语tf-idf权重------
-------这里输出第 25 类文本的词语tf-idf权重------
-------这里输出第 26 类文本的词语tf-idf权重------
-------这里输出

-------这里输出第 235 类文本的词语tf-idf权重------
-------这里输出第 236 类文本的词语tf-idf权重------
-------这里输出第 237 类文本的词语tf-idf权重------
-------这里输出第 238 类文本的词语tf-idf权重------
-------这里输出第 239 类文本的词语tf-idf权重------
-------这里输出第 240 类文本的词语tf-idf权重------
-------这里输出第 241 类文本的词语tf-idf权重------
-------这里输出第 242 类文本的词语tf-idf权重------
-------这里输出第 243 类文本的词语tf-idf权重------
-------这里输出第 244 类文本的词语tf-idf权重------
-------这里输出第 245 类文本的词语tf-idf权重------
-------这里输出第 246 类文本的词语tf-idf权重------
-------这里输出第 247 类文本的词语tf-idf权重------
-------这里输出第 248 类文本的词语tf-idf权重------
-------这里输出第 249 类文本的词语tf-idf权重------
-------这里输出第 250 类文本的词语tf-idf权重------
-------这里输出第 251 类文本的词语tf-idf权重------
-------这里输出第 252 类文本的词语tf-idf权重------
-------这里输出第 253 类文本的词语tf-idf权重------
-------这里输出第 254 类文本的词语tf-idf权重------
-------这里输出第 255 类文本的词语tf-idf权重------
-------这里输出第 256 类文本的词语tf-idf权重------
-------这里输出第 257 类文本的词语tf-idf权重------
-------这里输出第 258 类文本的词语tf-idf权重------
-------这里输出第 259 类文本的词语tf-idf权重------
-------这里输出第 260 类文本的词语tf-idf权重------
-------这里输出第

-------这里输出第 467 类文本的词语tf-idf权重------
-------这里输出第 468 类文本的词语tf-idf权重------
-------这里输出第 469 类文本的词语tf-idf权重------
-------这里输出第 470 类文本的词语tf-idf权重------
-------这里输出第 471 类文本的词语tf-idf权重------
-------这里输出第 472 类文本的词语tf-idf权重------
-------这里输出第 473 类文本的词语tf-idf权重------
-------这里输出第 474 类文本的词语tf-idf权重------
-------这里输出第 475 类文本的词语tf-idf权重------
-------这里输出第 476 类文本的词语tf-idf权重------
-------这里输出第 477 类文本的词语tf-idf权重------
-------这里输出第 478 类文本的词语tf-idf权重------
-------这里输出第 479 类文本的词语tf-idf权重------
-------这里输出第 480 类文本的词语tf-idf权重------
-------这里输出第 481 类文本的词语tf-idf权重------
-------这里输出第 482 类文本的词语tf-idf权重------
-------这里输出第 483 类文本的词语tf-idf权重------
-------这里输出第 484 类文本的词语tf-idf权重------
-------这里输出第 485 类文本的词语tf-idf权重------
-------这里输出第 486 类文本的词语tf-idf权重------
-------这里输出第 487 类文本的词语tf-idf权重------
-------这里输出第 488 类文本的词语tf-idf权重------
-------这里输出第 489 类文本的词语tf-idf权重------
-------这里输出第 490 类文本的词语tf-idf权重------
-------这里输出第 491 类文本的词语tf-idf权重------
-------这里输出第 492 类文本的词语tf-idf权重------
-------这里输出第

-------这里输出第 701 类文本的词语tf-idf权重------
-------这里输出第 702 类文本的词语tf-idf权重------
-------这里输出第 703 类文本的词语tf-idf权重------
-------这里输出第 704 类文本的词语tf-idf权重------
-------这里输出第 705 类文本的词语tf-idf权重------
-------这里输出第 706 类文本的词语tf-idf权重------
-------这里输出第 707 类文本的词语tf-idf权重------
-------这里输出第 708 类文本的词语tf-idf权重------
-------这里输出第 709 类文本的词语tf-idf权重------
-------这里输出第 710 类文本的词语tf-idf权重------
-------这里输出第 711 类文本的词语tf-idf权重------
-------这里输出第 712 类文本的词语tf-idf权重------
-------这里输出第 713 类文本的词语tf-idf权重------
-------这里输出第 714 类文本的词语tf-idf权重------
-------这里输出第 715 类文本的词语tf-idf权重------
-------这里输出第 716 类文本的词语tf-idf权重------
-------这里输出第 717 类文本的词语tf-idf权重------
-------这里输出第 718 类文本的词语tf-idf权重------
-------这里输出第 719 类文本的词语tf-idf权重------
-------这里输出第 720 类文本的词语tf-idf权重------
-------这里输出第 721 类文本的词语tf-idf权重------
-------这里输出第 722 类文本的词语tf-idf权重------
-------这里输出第 723 类文本的词语tf-idf权重------
-------这里输出第 724 类文本的词语tf-idf权重------
-------这里输出第 725 类文本的词语tf-idf权重------
-------这里输出第 726 类文本的词语tf-idf权重------
-------这里输出第

-------这里输出第 926 类文本的词语tf-idf权重------
-------这里输出第 927 类文本的词语tf-idf权重------
-------这里输出第 928 类文本的词语tf-idf权重------
-------这里输出第 929 类文本的词语tf-idf权重------
-------这里输出第 930 类文本的词语tf-idf权重------
-------这里输出第 931 类文本的词语tf-idf权重------
-------这里输出第 932 类文本的词语tf-idf权重------
-------这里输出第 933 类文本的词语tf-idf权重------
-------这里输出第 934 类文本的词语tf-idf权重------
-------这里输出第 935 类文本的词语tf-idf权重------
-------这里输出第 936 类文本的词语tf-idf权重------
-------这里输出第 937 类文本的词语tf-idf权重------
-------这里输出第 938 类文本的词语tf-idf权重------
-------这里输出第 939 类文本的词语tf-idf权重------
-------这里输出第 940 类文本的词语tf-idf权重------
-------这里输出第 941 类文本的词语tf-idf权重------
-------这里输出第 942 类文本的词语tf-idf权重------
-------这里输出第 943 类文本的词语tf-idf权重------
-------这里输出第 944 类文本的词语tf-idf权重------
-------这里输出第 945 类文本的词语tf-idf权重------
-------这里输出第 946 类文本的词语tf-idf权重------
-------这里输出第 947 类文本的词语tf-idf权重------
-------这里输出第 948 类文本的词语tf-idf权重------
-------这里输出第 949 类文本的词语tf-idf权重------
-------这里输出第 950 类文本的词语tf-idf权重------
-------这里输出第 951 类文本的词语tf-idf权重------
-------这里输出第

...

-------这里输出第 7388 类文本的词语tf-idf权重------
-------这里输出第 7389 类文本的词语tf-idf权重------
-------这里输出第 7390 类文本的词语tf-idf权重------
-------这里输出第 7391 类文本的词语tf-idf权重------
-------这里输出第 7392 类文本的词语tf-idf权重------
-------这里输出第 7393 类文本的词语tf-idf权重------
-------这里输出第 7394 类文本的词语tf-idf权重------
-------这里输出第 7395 类文本的词语tf-idf权重------
-------这里输出第 7396 类文本的词语tf-idf权重------
-------这里输出第 7397 类文本的词语tf-idf权重------
-------这里输出第 7398 类文本的词语tf-idf权重------
-------这里输出第 7399 类文本的词语tf-idf权重------
-------这里输出第 7400 类文本的词语tf-idf权重------
-------这里输出第 7401 类文本的词语tf-idf权重------
-------这里输出第 7402 类文本的词语tf-idf权重------
-------这里输出第 7403 类文本的词语tf-idf权重------
-------这里输出第 7404 类文本的词语tf-idf权重------
-------这里输出第 7405 类文本的词语tf-idf权重------
-------这里输出第 7406 类文本的词语tf-idf权重------
-------这里输出第 7407 类文本的词语tf-idf权重------
-------这里输出第 7408 类文本的词语tf-idf权重------
-------这里输出第 7409 类文本的词语tf-idf权重------
-------这里输出第 7410 类文本的词语tf-idf权重------
-------这里输出第 7411 类文本的词语tf-idf权重------
-------这里输出第 7412 类文本的词语tf-idf权重------
-------这里输出第 7413 类文本的词语t

-------这里输出第 7622 类文本的词语tf-idf权重------
-------这里输出第 7623 类文本的词语tf-idf权重------
-------这里输出第 7624 类文本的词语tf-idf权重------
-------这里输出第 7625 类文本的词语tf-idf权重------
-------这里输出第 7626 类文本的词语tf-idf权重------
-------这里输出第 7627 类文本的词语tf-idf权重------
-------这里输出第 7628 类文本的词语tf-idf权重------
-------这里输出第 7629 类文本的词语tf-idf权重------
-------这里输出第 7630 类文本的词语tf-idf权重------
-------这里输出第 7631 类文本的词语tf-idf权重------
-------这里输出第 7632 类文本的词语tf-idf权重------
-------这里输出第 7633 类文本的词语tf-idf权重------
-------这里输出第 7634 类文本的词语tf-idf权重------
-------这里输出第 7635 类文本的词语tf-idf权重------
-------这里输出第 7636 类文本的词语tf-idf权重------
-------这里输出第 7637 类文本的词语tf-idf权重------
-------这里输出第 7638 类文本的词语tf-idf权重------
-------这里输出第 7639 类文本的词语tf-idf权重------
-------这里输出第 7640 类文本的词语tf-idf权重------
-------这里输出第 7641 类文本的词语tf-idf权重------
-------这里输出第 7642 类文本的词语tf-idf权重------
-------这里输出第 7643 类文本的词语tf-idf权重------
-------这里输出第 7644 类文本的词语tf-idf权重------
-------这里输出第 7645 类文本的词语tf-idf权重------
-------这里输出第 7646 类文本的词语tf-idf权重------
-------这里输出第 7647 类文本的词语t

-------这里输出第 7840 类文本的词语tf-idf权重------
-------这里输出第 7841 类文本的词语tf-idf权重------
-------这里输出第 7842 类文本的词语tf-idf权重------
-------这里输出第 7843 类文本的词语tf-idf权重------
-------这里输出第 7844 类文本的词语tf-idf权重------
-------这里输出第 7845 类文本的词语tf-idf权重------
-------这里输出第 7846 类文本的词语tf-idf权重------
-------这里输出第 7847 类文本的词语tf-idf权重------
-------这里输出第 7848 类文本的词语tf-idf权重------
-------这里输出第 7849 类文本的词语tf-idf权重------
-------这里输出第 7850 类文本的词语tf-idf权重------
-------这里输出第 7851 类文本的词语tf-idf权重------
-------这里输出第 7852 类文本的词语tf-idf权重------
-------这里输出第 7853 类文本的词语tf-idf权重------
-------这里输出第 7854 类文本的词语tf-idf权重------
-------这里输出第 7855 类文本的词语tf-idf权重------
-------这里输出第 7856 类文本的词语tf-idf权重------
-------这里输出第 7857 类文本的词语tf-idf权重------
-------这里输出第 7858 类文本的词语tf-idf权重------
-------这里输出第 7859 类文本的词语tf-idf权重------
-------这里输出第 7860 类文本的词语tf-idf权重------
-------这里输出第 7861 类文本的词语tf-idf权重------
-------这里输出第 7862 类文本的词语tf-idf权重------
-------这里输出第 7863 类文本的词语tf-idf权重------
-------这里输出第 7864 类文本的词语tf-idf权重------
-------这里输出第 7865 类文本的词语t

-------这里输出第 8068 类文本的词语tf-idf权重------
-------这里输出第 8069 类文本的词语tf-idf权重------
-------这里输出第 8070 类文本的词语tf-idf权重------
-------这里输出第 8071 类文本的词语tf-idf权重------
-------这里输出第 8072 类文本的词语tf-idf权重------
-------这里输出第 8073 类文本的词语tf-idf权重------
-------这里输出第 8074 类文本的词语tf-idf权重------
-------这里输出第 8075 类文本的词语tf-idf权重------
-------这里输出第 8076 类文本的词语tf-idf权重------
-------这里输出第 8077 类文本的词语tf-idf权重------
-------这里输出第 8078 类文本的词语tf-idf权重------
-------这里输出第 8079 类文本的词语tf-idf权重------
-------这里输出第 8080 类文本的词语tf-idf权重------
-------这里输出第 8081 类文本的词语tf-idf权重------
-------这里输出第 8082 类文本的词语tf-idf权重------
-------这里输出第 8083 类文本的词语tf-idf权重------
-------这里输出第 8084 类文本的词语tf-idf权重------
-------这里输出第 8085 类文本的词语tf-idf权重------
-------这里输出第 8086 类文本的词语tf-idf权重------
-------这里输出第 8087 类文本的词语tf-idf权重------
-------这里输出第 8088 类文本的词语tf-idf权重------
-------这里输出第 8089 类文本的词语tf-idf权重------
-------这里输出第 8090 类文本的词语tf-idf权重------
-------这里输出第 8091 类文本的词语tf-idf权重------
-------这里输出第 8092 类文本的词语tf-idf权重------
-------这里输出第 8093 类文本的词语t

-------这里输出第 8281 类文本的词语tf-idf权重------
-------这里输出第 8282 类文本的词语tf-idf权重------
-------这里输出第 8283 类文本的词语tf-idf权重------
-------这里输出第 8284 类文本的词语tf-idf权重------
-------这里输出第 8285 类文本的词语tf-idf权重------
-------这里输出第 8286 类文本的词语tf-idf权重------
-------这里输出第 8287 类文本的词语tf-idf权重------
-------这里输出第 8288 类文本的词语tf-idf权重------
-------这里输出第 8289 类文本的词语tf-idf权重------


In [23]:
term_weights.sort(key=lambda v:v[1], reverse=True)

In [24]:
term_weights

[('刹车', 1.0),
 ('配置', 1.0),
 ('价格', 1.0),
 ('好换', 1.0),
 ('空调', 1.0),
 ('机油', 1.0),
 ('导航', 1.0),
 ('油耗', 1.0),
 ('动力', 1.0),
 ('变速箱', 1.0),
 ('优惠', 1.0),
 ('刹车', 1.0),
 ('优惠', 1.0),
 ('做工', 1.0),
 ('油耗', 1.0),
 ('优惠', 1.0),
 ('地力', 1.0),
 ('动力', 1.0),
 ('后备箱', 1.0),
 ('刹车盘', 1.0),
 ('好看', 1.0),
 ('动力', 1.0),
 ('配置', 1.0),
 ('油耗', 1.0),
 ('油耗', 1.0),
 ('变速箱', 1.0),
 ('声音', 1.0),
 ('油耗', 1.0),
 ('车漆', 1.0),
 ('空调', 1.0),
 ('异响', 1.0),
 ('价格', 1.0),
 ('排量', 1.0),
 ('油耗', 1.0),
 ('价格', 1.0),
 ('动力', 1.0),
 ('机油', 1.0),
 ('座椅', 1.0),
 ('响动', 1.0),
 ('油耗', 1.0),
 ('油耗', 1.0),
 ('5000km', 1.0),
 ('油耗', 1.0),
 ('换油', 1.0),
 ('机油', 1.0),
 ('机油', 1.0),
 ('价格', 1.0),
 ('急刹车', 1.0),
 ('九成新', 0.9496282949176235),
 ('年里', 0.9496282949176235),
 ('导航仪', 0.9491266692944575),
 ('0l', 0.9471743039296961),
 ('行货', 0.9452678304391573),
 ('带行', 0.9452678304391573),
 ('太会', 0.9452678304391573),
 ('礼品', 0.9452678304391573),
 ('怕怕', 0.9450586522128102),
 ('烧烧', 0.9450586522128102),
 ('东假', 0.9450586522128102)

In [25]:
len(term_weights)

79049

In [26]:
import re
alphanum = re.compile('[a-zA-Z0-9]')

In [27]:
important_words = set(v[0] for v, _ in zip(term_weights, range(3000)) if not alphanum.search(v[0]))

In [28]:
important_words

{'圆滑',
 '持平',
 '买贵',
 '稳定',
 '霸道',
 '吸出来',
 '合肥',
 '外形',
 '占优势',
 '购进',
 '光线',
 '变速箱',
 '力度',
 '溢流',
 '注意安全',
 '小毛病',
 '年底',
 '温度',
 '车架号',
 '听诊器',
 '夏天',
 '全系',
 '高配',
 '加油站',
 '确实',
 '昂科威',
 '拉到',
 '强劲',
 '可想而知',
 '省油',
 '十一万',
 '山顶',
 '透露',
 '息火',
 '驾驭',
 '踩油门',
 '但用',
 '听过',
 '一款',
 '只卖',
 '换不换',
 '卖点',
 '资料',
 '挺大',
 '狮特',
 '开启',
 '时速',
 '管它',
 '风格',
 '车标',
 '三代',
 '用拓乐',
 '频繁',
 '强化',
 '声音',
 '怀疑',
 '风一吹会',
 '听说',
 '专家',
 '算得',
 '气缸',
 '漆磨',
 '升挡',
 '往前',
 '车烧',
 '兄是',
 '不上',
 '那款',
 '切记',
 '石墨',
 '花钱买',
 '有带',
 '吃亏',
 '土罐',
 '开开',
 '好丑',
 '全市',
 '正宗',
 '底线',
 '干练',
 '完换',
 '过后',
 '太响',
 '机盖',
 '济南',
 '更新',
 '螺母',
 '多用',
 '材料',
 '一批',
 '没治',
 '车道',
 '看烧',
 '轻点',
 '解答',
 '装没装',
 '怕怕',
 '车轮',
 '换成',
 '暴力',
 '不划算',
 '外观',
 '锻造',
 '再换',
 '行程',
 '三万',
 '车型',
 '扇叶',
 '挫样',
 '优惠价格',
 '销售价格',
 '翻修',
 '改动',
 '明天',
 '低频',
 '两边',
 '我想海',
 '就别',
 '龙华',
 '大脚',
 '打理',
 '差价',
 '比一比',
 '款不烧',
 '相对来说',
 '说得对',
 '车顶',
 '地开',
 '测速',
 '森好',
 '捷达',
 '冲量',
 '五六年',
 '说实话',
 '高速行驶',
 '不知',
 '无烧',
 '归零',

In [29]:
len(important_words)

2179

In [30]:
with open('../data/tfidf.txt', 'w', encoding='utf-8') as f:
    first = True
    for w in important_words:
        if first:
            first = False
        else:
            f.write('\n')
        f.write(w)
    

In [31]:
tdf = pd.DataFrame()
#tdf['content'] = df['content']
#tdf['content_id'] = df['content_id']
for w in important_words:
    tdf[w] = df['content'].apply(lambda text: 1 if text.find(w) != -1 else 0)

In [32]:
tdf.shape

(8290, 2179)

In [33]:
tdf.to_csv('../data/tfidf_train.csv', index=False)